In [1]:
'''
Task 2: Real-Time Stream Simulation & Event Trigger
·Simulate a video stream using OpenCV
·Perform detection on every 3rd frame
·Trigger an alert (e.g., "Crowd Detected") if 3+ people appear in 5 consecutive frames
·Log alerts in .json or .txt format
·Plot a timeline of alert occurrences
·(Optional) Overlay alerts and export the output visuals
'''
import cv2
import torch
import json
import datetime
import matplotlib.pyplot as plt

# Load YOLOv5 model (better accuracy) and move to GPU if available
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')
model.classes = [0]  # Detect only persons
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

#1. "C:\\Users\\Randhir kumar\\Downloads\\2048206-hd_1920_1080_30fps.mp4"
#2. "C:\\Users\\Randhir kumar\\Downloads\\7334800-hd_1920_1080_24fps.mp4"

cap = cv2.VideoCapture(r"C:\\Users\\Randhir kumar\\Downloads\\7334800-hd_1920_1080_24fps.mp4")

frame_count=0
recent_counts=[]
alerts=[]
output_frames=[]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    alert_triggered = False

    frame = cv2.resize(frame, (960, 540))  # Resize for speed

    if frame_count % 3 == 0:
        results = model(frame)  # ✅ FIXED
        detections = results.xyxy[0].cpu()

        # Count people and draw boxes
        count = 0
        for *box, conf, cls in detections:
            if int(cls) == 0:
                count += 1
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Update recent person counts
        recent_counts.append(count)
        if len(recent_counts) > 5:
            recent_counts.pop(0)

        # Check alert condition
        if len(recent_counts) == 5 and all(c >= 3 for c in recent_counts):
            alert_triggered = True
            alerts.append({
                "time": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "alert": "Crowd Detected"
            })

        # Draw labels
        cv2.putText(frame, f"People: {count}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        if alert_triggered:
            cv2.putText(frame, "ALERT: Crowd Detected!", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    output_frames.append(frame)
    cv2.imshow("Stream", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save alerts to JSON
with open("alerts.json", "w") as f:
    json.dump(alerts, f, indent=2)

# Plot alert timeline
if alerts:
    times = [datetime.datetime.strptime(a["time"], "%Y-%m-%d %H:%M:%S") for a in alerts]
    plt.plot(times, range(1, len(times) + 1), marker='o')
    plt.title("Alert Timeline")
    plt.xlabel("Time")
    plt.ylabel("Alert Count")
    plt.grid(True)
    plt.savefig("alert_timeline.png")
    plt.show()

# Save video with overlays
if output_frames:
    h, w, _ = output_frames[0].shape
    out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"XVID"), 20, (w, h))
    for frame in output_frames:
        out.write(frame)
    out.release()

Using cache found in C:\Users\Randhir kumar/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-7-12 Python-3.12.0 torch-2.7.1+cpu CPU

100%|██████████████████████████████████████████████████████████████████████████████| 40.8M/40.8M [01:10<00:00, 611kB/s]

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 
